### Note: This notebook was run on my local machine and the results are displayed here. The code cannot be run in Colab.

In [1]:
import jax.numpy as jnp

from src.mcnnm.wrappers import estimate, complete_matrix
from src.mcnnm.utils import generate_data

## Example 1: Staggered Adoption with Cross-Validation (Default)
In this example, we generate a dataset with covariates and staggered adoption treatment assignment and use the default cross-validation method for selecting the regularization parameters. Cross-validation is currently not parallelized and may take longer to run on large datasets or smaller processors.

In [6]:
Y, W, X, Z, V, true_params = generate_data(
        nobs=50,
        nperiods=10,
        unit_fe=True,
        time_fe=True,
        X_cov=True,
        Z_cov=True,
        V_cov=True,
        seed=2024,
        noise_scale=0.2,
        autocorrelation=0.0,
        assignment_mechanism="last_periods",
        treated_fraction=0.4,
        last_treated_periods=3,
    )

# Run estimation
results = estimate(
    Y=Y,
    W=W,
    X=X,
    Z=Z,
    V=V,
    Omega=None,
    use_unit_fe=True,
    use_time_fe=True,
    lambda_L=None,
    lambda_H=None,
    validation_method='cv',  # type: ignore
    K=10,  # Use 2 folds for faster testing
    n_lambda=12,  # Use 3 lambda values for faster testing
    max_iter=1e5,  # 12
    tol=1e-5,  # 14
)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.12f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")


True effect: 5.0, Estimated effect: 5.272798700257
Chosen lambda_L: 0.0000, lambda_H: 0.1806


The `generate_data` function is used to create a synthetic dataset with staggered adoption treatment assignment. The assignment_mechanism parameter is set to `staggered`, which means that each unit adopts the treatment at a random time point with a specified probability.
By default, the estimate function uses cross-validation to select the optimal regularization parameters lambda_L and lambda_H. Cross-validation splits the data into K folds (default is 5) and evaluates the model performance on each fold to select the best parameters.

## Example 2: Block Assignment with Holdout Validation
In this example, we generate a dataset without covariates using block treatment assignment and use holdout validation for selecting the regularization parameters.

In [3]:


results = estimate(Y, W, validation_method='holdout')

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.4f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")

ValueError: Holdout validation requires initial_window, step_size, and horizon.

Here, the `assignment_mechanism` is set to `block`, which means that a specified fraction of units (determined by `treated_fraction`) are treated in the second half of the time periods.
The validation_method parameter in the estimate function is set to `holdout`, indicating that holdout validation should be used for selecting the regularization parameters. Holdout validation splits the data into a training set and a validation set based on time. It uses the earlier time periods for training and the later time periods for validation. Holdout validation is typically faster than cross-validation but may be less accurate, especially if the number of time periods is small.

## Example 3: Single Treated Unit with Covariates
In this example, we generate a dataset with a single treated unit and include covariates in the estimation.

In [ ]:
data, true_params = generate_data(nobs=100, nperiods=200, seed=456, assignment_mechanism='single_treated_unit', 
                                  X_cov=True, Z_cov=True, V_cov=True)

Y = jnp.array(data.pivot(index='unit', columns='period', values='y').values)
W = jnp.array(data.pivot(index='unit', columns='period', values='treat').values)
X = jnp.array(true_params['X'])
Z = jnp.array(true_params['Z'])
V = jnp.array(true_params['V'])

results = estimate(Y, W, X=X, Z=Z, V=V, K=3)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.4f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")

The `assignment_mechanism` is set to `'single_treated_unit'`, which means that only one randomly selected unit is treated in the second half of the time periods.

In this example, we include unit-specific covariates `X`, time-specific covariates `Z`, and unit-time specific covariates `V` in the estimation. The `estimate` function automatically handles the presence of covariates and estimates their coefficients along with the treatment effect.

With this specific dataset, the estimated treatment effect is not close to the true treatment effect, as the single treated unit leads to the cross-validation method struggling to find a valid loss during the parameter selection process. The warning message "No valid loss found in cross_validate" indicates that the cross-validation procedure could not find a suitable set of regularization parameters that yielded a finite loss value.

This issue arises because with only a single treated unit, there might not be enough information to reliably estimate the treatment effect, especially when using cross-validation. The limited treatment variation can make it challenging for the model to distinguish the treatment effect from the noise in the data.

In such cases, it may be more appropriate to use a different validation method, such as holdout validation, or to rely on domain knowledge to set the regularization parameters manually. Additionally, increasing the number of observations or treated units can help improve the estimation accuracy and stability.

It's important to note that the performance of the estimation method can be sensitive to the specific dataset and the chosen assignment mechanism. While the `estimate` function aims to handle various scenarios, there may be limitations in extreme cases like having only a single treated unit. It's always a good practice to carefully evaluate the results, consider the characteristics of the dataset, and interpret the findings in the context of the specific application.

## Example 4: Custom Lambda Grid with Cross-Validation

In this example, we demonstrate how to use  custom lambda values for cross-validation. As the `n_lambda_L` and `n_lambda_H` parameters are set to 3, the cross-validation method will select the best lambda values from around the specified values.

In [ ]:
data, true_params = generate_data(nobs=100, nperiods=50, seed=789, assignment_mechanism='staggered')

Y = jnp.array(data.pivot(index='unit', columns='period', values='y').values)
W = jnp.array(data.pivot(index='unit', columns='period', values='treat').values)

results = estimate(Y, W, lambda_L=0.1, n_lambda_L=3, lambda_H=0.01, n_lambda_H=3)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.4f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")

## Example 5: Estimation with Holdout Validation
In this example, we generate data with staggered adoption treatment assignment and use holdout validation for selecting the regularization parameters. The `max_window_size` parameter is set to 80, which controls the maximum size of the training window in the holdout validation process.

In [ ]:
data, true_params = generate_data(nobs=200, nperiods=100, seed=101, assignment_mechanism='staggered')

Y = jnp.array(data.pivot(index='unit', columns='period', values='y').values)
W = jnp.array(data.pivot(index='unit', columns='period', values='treat').values)

results = estimate(Y, W, validation_method='holdout', max_window_size=80)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.4f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")

## Example 6: Estimation with Pre-specified Lambda Values

This example shows how to estimate the model using pre-specified lambda values.

In [ ]:
data, true_params = generate_data(nobs=150, nperiods=75, seed=202, assignment_mechanism='staggered')

Y = jnp.array(data.pivot(index='unit', columns='period', values='y').values)
W = jnp.array(data.pivot(index='unit', columns='period', values='treat').values)

results = estimate(Y, W, lambda_L=0.05, lambda_H=0.01)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.4f}")
print(f"Used lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")

## Example 7: Estimation with Autocorrelation

In this example, we generate data with autocorrelation (see section 8.3 of [Athey et al. (2021)](https://www.tandfonline.com/doi/full/10.1080/01621459.2021.1891924)) and use a custom Omega matrix in estimation.

In [ ]:
data, true_params = generate_data(nobs=120, nperiods=60, seed=303, assignment_mechanism='staggered', autocorrelation=0.5)

Y = jnp.array(data.pivot(index='unit', columns='period', values='y').values)
W = jnp.array(data.pivot(index='unit', columns='period', values='treat').values)

# Create custom Omega matrix with AR(1) structure
rho = 0.5
T = Y.shape[1]
Omega = jnp.power(rho, jnp.abs(jnp.arange(T)[:, None] - jnp.arange(T)))

results = estimate(Y, W, Omega=Omega)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.4f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")

## Example 8: Matrix Completion

This example demonstrates how to use the `complete_matrix()` function to impute missing values.

In [ ]:
data, true_params = generate_data(nobs=200, nperiods=40, seed=404, treatment_probability=0.2)

Y = jnp.array(data.pivot(index='unit', columns='period', values='y').values)
W = jnp.array(data.pivot(index='unit', columns='period', values='treat').values)  # randomly mask out entries

results = complete_matrix(Y, W)

print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")
print(f"Mean absolute error of imputation: {jnp.mean(jnp.abs(Y - results.Y_completed)):.4f}")

## Example 9: Different Treatment Assignment Mechanisms

This example showcases the 'single_treated_period' and 'last_periods' assignment mechanisms.

In [ ]:
# Single treated period
data_single, true_params_single = generate_data(nobs=200, nperiods=100, seed=505, 
                                                assignment_mechanism='single_treated_period')

Y_single = jnp.array(data_single.pivot(index='unit', columns='period', values='y').values)
W_single = jnp.array(data_single.pivot(index='unit', columns='period', values='treat').values)

results_single = estimate(Y_single, W_single)

print("Single Treated Period:")
print(f"True effect: {true_params_single['treatment_effect']}, Estimated effect: {results_single.tau:.4f}")

# Last periods treated
data_last, true_params_last = generate_data(nobs=200, nperiods=100, seed=606, 
                                            assignment_mechanism='last_periods', last_treated_periods=20)

Y_last = jnp.array(data_last.pivot(index='unit', columns='period', values='y').values)
W_last = jnp.array(data_last.pivot(index='unit', columns='period', values='treat').values)

results_last = estimate(Y_last, W_last)

print("\nLast Periods Treated:")
print(f"True effect: {true_params_last['treatment_effect']}, Estimated effect: {results_last.tau:.4f}")

# Covariates
The `generate_data` function allows you to include three types of covariates in the generated dataset:

1. **Unit-specific covariates (X):** These are characteristics or features that vary across units but remain constant over time. For example, in a study of students' academic performance, unit-specific covariates could include variables like gender, age, or socioeconomic status. These covariates capture the inherent differences between units that may influence the outcome variable.
2. **Time-specific covariates (Z):** These are factors that change over time but are the same for all units at each time point. For instance, in an analysis of sales data, time-specific covariates could include variables like market trends, seasonal effects, or economic indicators. These covariates reflect the temporal variations that affect all units simultaneously.
3. **Unit-time specific covariates (V):** These are covariates that vary both across units and over time. They capture the unique characteristics of each unit at each time point. For example, in a healthcare study, unit-time specific covariates could include individual patients' medical measurements or treatment adherence recorded at different time points. These covariates allow for capturing the dynamic and personalized aspects of each unit's experience.

These three options are available for estimation, mirroring the description of the estimator in [Athey et al. (2021)](https://www.tandfonline.com/doi/full/10.1080/01621459.2021.1891924).

In the `generate_data` function, you can control the inclusion of these covariates using the boolean flags X_cov, Z_cov, and V_cov. Setting these flags to True incorporates the respective type of covariates into the generated dataset, while setting them to False excludes them.